**Preparation of customer dimensional table**

In [0]:
query = """
SELECT
    ROW_NUMBER() OVER (ORDER BY ci.customer_id) AS customer_key,
    ci.customer_id,
    ci.customer_number,
    ci.first_name,
    ci.last_name,
    la.country,
    ci.marital_status,
    CASE
        WHEN ci.gender <> 'n/a' THEN ci.gender
        ELSE COALESCE(ca.gender, 'n/a')
    END AS gender,
    ca.birth_date AS birthdate,
    ci.created_date AS create_date
FROM workspace.silver.crm_cust_info ci
LEFT JOIN workspace.silver.erp_customers ca
    ON ci.customer_number = ca.customer_number
LEFT JOIN workspace.silver.erp_customer_location la
    ON ci.customer_number = la.customer_number
"""
df = spark.sql(query)

**Write into gold table**

In [0]:
spark.sql("DROP TABLE IF EXISTS gold.dim_customers")
df.write.mode("overwrite").format("delta").saveAsTable("gold.dim_customers")